In [11]:
#!/usr/bin/env python3
# ------------------------------------------------------------
# purpose: Build greenspace indices (equal weights, NLCD schemes, PCA)
#          and validate against CDC PLACES outcomes (pooled + RUCA strat)
# notes:   Python 3.9 compatible; robust scaling + NLCD de-dupe
# ------------------------------------------------------------

import re
import warnings
from pathlib import Path
from typing import Optional, Dict, List, Tuple

import numpy as np
import pandas as pd

import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

warnings.filterwarnings("ignore", category=RuntimeWarning)


# ============================================================
# 0) CONFIG
# ============================================================

ROOT = Path("/mnt/sda-21.8/bdevoe/greenspace")

NDVI_FILE = ROOT / "processed" / "tract_ndvi_summer_2021_2025_summary.csv"
EVI_FILE  = ROOT / "processed" / "tract_evi_summer_2021_2025_summary.csv"
TCC_FILE  = ROOT / "processed" / "tract_tcc_nlcd_2021_2023_summary.csv"

NLCD_FILE   = ROOT / "processed" / "nlcd_2023_tract_fullclass_pct_cb2020_500k_CONUS_READABLE.csv"
PLACES_FILE = ROOT / "outcomes" / "places_new" / "places_tracts.csv"

RUCA_FILE = None

GEOID_CANDIDATES = ["GEOID", "geoid", "TractFIPS", "tractfips", "FIPS", "fips"]

NDVI_COL_CANDIDATES = ["mean_ndvi", "ndvi_mean", "ndvi", "NDVI"]
EVI_COL_CANDIDATES  = ["mean_evi", "evi_mean", "evi", "EVI"]
TCC_COL_CANDIDATES  = [
    "tcc_mean_2021_2023",
    "tcc_2023",
    "mean_tcc",
    "tcc_mean",
    "tcc",
    "TCC",
    "tree_canopy",
    "pct_tree_canopy",
]

STATE_COL_CANDIDATES = ["StateAbbr", "state", "STATE", "stusps", "STUSPS"]
RUCA_COL_CANDIDATES  = ["ruca10", "RUCA10", "ruca", "RUCA", "ruca_code"]

PLACES_OUTCOMES = [
    "OBESITY_CrudePrev",
    "LPA_CrudePrev",
    "MHLTH_CrudePrev",
    "PHLTH_CrudePrev",
]

WINSORIZE = True
WINSOR_Q = (0.01, 0.99)
EPS = 1e-12

SAVE_OUTPUTS = True
OUT_DIR = ROOT / "processed" / "index_validation"
OUT_DIR.mkdir(parents=True, exist_ok=True)


# ============================================================
# 1) HELPERS
# ============================================================

def find_first_existing_col(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    for c in candidates:
        if c in df.columns:
            return c
    return None

def standardize_geoid(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip()
    s = s.str.replace(r"\.0$", "", regex=True)
    s = s.str.replace(r"[^0-9]", "", regex=True)
    s = s.str.zfill(11)
    return s

def winsorize_series(x: pd.Series, q_low: float = 0.01, q_high: float = 0.99) -> pd.Series:
    """
    Robust winsorization:
    - coerce to numeric
    - compute quantiles
    - use np.clip (avoids pandas alignment / axis errors)
    """
    x = pd.to_numeric(x, errors="coerce")
    lo = x.quantile(q_low)
    hi = x.quantile(q_high)
    arr = x.to_numpy(dtype=float)
    return pd.Series(np.clip(arr, lo, hi), index=x.index)

def minmax_01(x: pd.Series) -> pd.Series:
    x = pd.to_numeric(x, errors="coerce").astype(float)
    xmin = np.nanmin(x.values)
    xmax = np.nanmax(x.values)
    if np.isfinite(xmin) and np.isfinite(xmax) and (xmax - xmin) > EPS:
        return (x - xmin) / (xmax - xmin)
    return pd.Series(np.nan, index=x.index)

def scale_01(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    out = df.copy()
    for c in cols:
        if c not in out.columns:
            continue
        x = pd.to_numeric(out[c], errors="coerce")
        if WINSORIZE:
            x = winsorize_series(x, WINSOR_Q[0], WINSOR_Q[1])
        out[f"{c}_01"] = minmax_01(x)
    return out

def safe_mean(df: pd.DataFrame, cols: List[str], outcol: str) -> pd.DataFrame:
    df[outcol] = df[cols].mean(axis=1, skipna=True)
    return df

def print_basic_diagnostics(df: pd.DataFrame, cols: List[str], title: str) -> None:
    print("\n" + "="*90)
    print(title)
    print("="*90)
    for c in cols:
        if c not in df.columns:
            print(f"- {c}: MISSING")
            continue
        x = pd.to_numeric(df[c], errors="coerce")
        n = x.notna().sum()
        if n == 0:
            print(f"- {c}: all missing")
            continue
        print(
            f"- {c}: n={n:,}  "
            f"min={np.nanmin(x):.4f}  p1={x.quantile(0.01):.4f}  "
            f"p50={x.quantile(0.50):.4f}  p99={x.quantile(0.99):.4f}  "
            f"max={np.nanmax(x):.4f}"
        )

def load_indicator(file_path: Path, value_candidates: List[str]) -> Tuple[pd.DataFrame, str]:
    df = pd.read_csv(file_path)

    geoid_col = find_first_existing_col(df, GEOID_CANDIDATES)
    if geoid_col is None:
        raise ValueError(f"No GEOID column found in {file_path.name}. Columns: {list(df.columns)[:50]} ...")

    val_col = find_first_existing_col(df, value_candidates)
    if val_col is None:
        raise ValueError(f"No indicator value column found in {file_path.name}. Columns: {list(df.columns)[:50]} ...")

    df = df[[geoid_col, val_col]].copy()
    df.rename(columns={geoid_col: "GEOID"}, inplace=True)
    df["GEOID"] = standardize_geoid(df["GEOID"])
    return df, val_col

def normalize_weights(w: Dict[str, float]) -> Dict[str, float]:
    s = float(sum(w.values()))
    if s <= 0:
        raise ValueError("Weights must sum to > 0.")
    return {k: float(v) / s for k, v in w.items()}

def build_weighted_index(df: pd.DataFrame, weights: Dict[str, float], outcol: str) -> pd.DataFrame:
    keys = list(weights.keys())
    w = np.array([weights[k] for k in keys], dtype=float)

    X = df[keys].to_numpy(dtype=float)
    mask = np.isfinite(X)

    w_mat = np.tile(w, (X.shape[0], 1))
    w_mat[~mask] = 0.0

    w_sum = w_mat.sum(axis=1)
    w_sum = np.where(w_sum > EPS, w_sum, np.nan)

    df[outcol] = np.nansum(X * w_mat, axis=1) / w_sum
    return df

def run_models(
    df: pd.DataFrame,
    outcome: str,
    index_col: str,
    state_col: str,
    ruca_col: Optional[str],
) -> pd.DataFrame:
    results = []

    cols = [outcome, index_col, state_col]
    if ruca_col:
        cols.append(ruca_col)

    d = df[cols].copy().dropna(subset=[outcome, index_col, state_col])

    m1 = smf.ols(f"{outcome} ~ {index_col}", data=d).fit(cov_type="HC1")
    results.append(("pooled_unadjusted", None, m1))

    m2 = smf.ols(f"{outcome} ~ {index_col} + C({state_col})", data=d).fit(cov_type="HC1")
    results.append(("pooled_stateFE", None, m2))

    if ruca_col and ruca_col in d.columns:
        m3 = smf.ols(f"{outcome} ~ {index_col} + C({state_col}) + C({ruca_col})", data=d).fit(cov_type="HC1")
        results.append(("pooled_stateFE_rucaFE", None, m3))

        for r in sorted(d[ruca_col].dropna().unique()):
            dr = d[d[ruca_col] == r].copy()
            if len(dr) < 200:
                continue
            ma = smf.ols(f"{outcome} ~ {index_col}", data=dr).fit(cov_type="HC1")
            mb = smf.ols(f"{outcome} ~ {index_col} + C({state_col})", data=dr).fit(cov_type="HC1")
            results.append(("ruca_strat_unadjusted", r, ma))
            results.append(("ruca_strat_stateFE", r, mb))

    tidy = []
    for model_name, ruca_level, model in results:
        if index_col not in model.params.index:
            continue
        tidy.append({
            "index": index_col,
            "model": model_name,
            "ruca_level": ruca_level,
            "outcome": outcome,
            "estimate": float(model.params[index_col]),
            "std_error": float(model.bse[index_col]),
            "t": float(model.tvalues[index_col]),
            "p": float(model.pvalues[index_col]),
            "n": int(model.nobs),
            "r2": float(model.rsquared),
        })
    return pd.DataFrame(tidy)


# ============================================================
# 2) LOAD + MERGE
# ============================================================

print("\nLoading indicators...")
ndvi_df, ndvi_col = load_indicator(NDVI_FILE, NDVI_COL_CANDIDATES)
evi_df,  evi_col  = load_indicator(EVI_FILE,  EVI_COL_CANDIDATES)
tcc_df,  tcc_col  = load_indicator(TCC_FILE,  TCC_COL_CANDIDATES)

print(f"  NDVI: {NDVI_FILE.name}  col={ndvi_col}  rows={len(ndvi_df):,}")
print(f"  EVI : {EVI_FILE.name}   col={evi_col}   rows={len(evi_df):,}")
print(f"  TCC : {TCC_FILE.name}   col={tcc_col}   rows={len(tcc_df):,}")

bio = ndvi_df.merge(evi_df, on="GEOID", how="outer").merge(tcc_df, on="GEOID", how="outer")
print(f"Biophysical merged rows: {len(bio):,}")

print("\nLoading NLCD + PLACES...")
nlcd = pd.read_csv(NLCD_FILE)
nlcd_geoid = find_first_existing_col(nlcd, GEOID_CANDIDATES)
if nlcd_geoid is None:
    raise ValueError(f"No GEOID in NLCD file. Columns: {list(nlcd.columns)[:50]} ...")
nlcd = nlcd.rename(columns={nlcd_geoid: "GEOID"})
nlcd["GEOID"] = standardize_geoid(nlcd["GEOID"])

places = pd.read_csv(PLACES_FILE)
places_geoid = find_first_existing_col(places, GEOID_CANDIDATES)
if places_geoid is None:
    raise ValueError(f"No GEOID in PLACES file. Columns: {list(places.columns)[:50]} ...")
places = places.rename(columns={places_geoid: "GEOID"})
places["GEOID"] = standardize_geoid(places["GEOID"])

state_col = find_first_existing_col(places, STATE_COL_CANDIDATES)
if state_col is None:
    raise ValueError(f"No state column found in PLACES. Tried: {STATE_COL_CANDIDATES}")
print(f"Using state col: {state_col}")

ruca_col = find_first_existing_col(places, RUCA_COL_CANDIDATES)
if ruca_col:
    print(f"RUCA found in PLACES: {ruca_col}")
else:
    ruca_col = None
    print("No RUCA detected in PLACES (pooled models only).")

# ============================================================
# 3) PICK NLCD COLUMNS (READABLE name-based)
# ============================================================

nlcd_cols = list(nlcd.columns)

def pick_cols_by_regex(patterns: List[str]) -> List[str]:
    picked = []
    for pat in patterns:
        rx = re.compile(pat, flags=re.IGNORECASE)
        picked.extend([c for c in nlcd_cols if rx.search(c)])
    return picked  # keep order-ish; we will de-dupe later

forest_cols   = pick_cols_by_regex([r"forest", r"decidu", r"evergr", r"mixed"])
shrub_cols    = pick_cols_by_regex([r"shrub"])
grass_cols    = pick_cols_by_regex([r"grass", r"herb"])
wetland_cols  = pick_cols_by_regex([r"wetland", r"woody", r"emerg"])
pasture_cols  = pick_cols_by_regex([r"pasture", r"hay"])
open_cols     = pick_cols_by_regex([r"open[_ ]?space", r"dev.*open"])

NLCD_SELECTED = forest_cols + shrub_cols + grass_cols + wetland_cols + pasture_cols + open_cols

# De-duplicate while preserving order
NLCD_SELECTED = list(dict.fromkeys(NLCD_SELECTED))

if len(NLCD_SELECTED) == 0:
    print("\nWARNING: Could not auto-detect NLCD columns by name.")
    print("NLCD columns preview (first 150):")
    print(nlcd.columns.tolist()[:150])
    raise ValueError("Set NLCD_SELECTED manually (no NLCD columns detected).")

print("\nNLCD columns selected:")
for c in NLCD_SELECTED:
    print(f"  - {c}")

# ============================================================
# 4) MERGE ALL
# ============================================================

df = bio.merge(nlcd[["GEOID"] + NLCD_SELECTED], on="GEOID", how="left")
df = df.merge(places, on="GEOID", how="left")

print(f"\nFull merged rows: {len(df):,}")
print(f"Rows with PLACES present ({PLACES_OUTCOMES[0]}): {pd.to_numeric(df[PLACES_OUTCOMES[0]], errors='coerce').notna().sum():,}")

# ============================================================
# 5) SCALE TO [0,1]
# ============================================================

df = scale_01(df, [ndvi_col, evi_col, tcc_col])
df = scale_01(df, NLCD_SELECTED)

scaled_preview = [f"{ndvi_col}_01", f"{evi_col}_01", f"{tcc_col}_01"] + [f"{c}_01" for c in NLCD_SELECTED]
print_basic_diagnostics(df, scaled_preview[:12], title="Scaled diagnostics (first 12 scaled cols)")

# ============================================================
# 6) BUILD INDICES
# ============================================================

df = safe_mean(df, [f"{ndvi_col}_01", f"{evi_col}_01", f"{tcc_col}_01"], "green_eq")

base_components = {f"{ndvi_col}_01": 1.0, f"{evi_col}_01": 1.0, f"{tcc_col}_01": 1.0}
nlcd_components = {f"{c}_01": 1.0 for c in NLCD_SELECTED}

def make_scheme(name: str, w_base: float, w_nlcd: float, extra: Optional[Dict[str, float]] = None):
    w: Dict[str, float] = {}
    w.update({k: v * w_base for k, v in base_components.items()})
    w.update({k: v * w_nlcd for k, v in nlcd_components.items()})
    if extra:
        w.update(extra)
    return name, normalize_weights(w)

schemes = [
    make_scheme("nlcd_eq_expanded",      1.0, 1.0),
    make_scheme("nlcd_bio_dominant",     3.0, 1.0),
    make_scheme("nlcd_canopy_weighted",  1.0, 1.0, extra={f"{tcc_col}_01": 2.0}),
    make_scheme("nlcd_vi_weighted",      1.0, 1.0, extra={f"{ndvi_col}_01": 2.0, f"{evi_col}_01": 2.0}),
    make_scheme("nlcd_nlcd_dominant",    1.0, 3.0),
    make_scheme("nlcd_conservative",     4.0, 1.0),
    make_scheme("nlcd_balanced_canopy",  2.0, 2.0, extra={f"{tcc_col}_01": 1.5}),
]

for name, w in schemes:
    df = build_weighted_index(df, w, outcol=f"green_{name}")

# PCA PC1
pca_inputs = [f"{ndvi_col}_01", f"{evi_col}_01", f"{tcc_col}_01"] + [f"{c}_01" for c in NLCD_SELECTED]
pca_df = df[pca_inputs].dropna(axis=0, how="any")
df["green_pca_pc1"] = np.nan

if len(pca_df) >= 1000:
    scaler = StandardScaler()
    Xz = scaler.fit_transform(pca_df.values)

    pca = PCA(n_components=3, random_state=42)
    pcs = pca.fit_transform(Xz)

    df.loc[pca_df.index, "green_pca_pc1"] = pcs[:, 0]

    corr = df.loc[pca_df.index, "green_pca_pc1"].corr(df.loc[pca_df.index, "green_eq"])
    if corr is not None and corr < 0:
        df.loc[pca_df.index, "green_pca_pc1"] *= -1

    print("\nPCA summary:")
    print(f"  N used: {len(pca_df):,}")
    print(f"  Explained variance ratios: {pca.explained_variance_ratio_}")

    loadings = pd.Series(pca.components_[0], index=pca_inputs).sort_values(ascending=False)
    print("\n  PC1 loadings (sorted):")
    print(loadings.to_string())
else:
    print("\nWARNING: Not enough complete rows for PCA (need >= 1000). PCA skipped.")

# ============================================================
# 7) VALIDATION REGRESSIONS
# ============================================================

missing_outcomes = [o for o in PLACES_OUTCOMES if o not in df.columns]
if missing_outcomes:
    raise ValueError(f"Missing PLACES outcomes in merged df: {missing_outcomes}")

index_cols = ["green_eq"] + [f"green_{name}" for name, _ in schemes] + ["green_pca_pc1"]

print("\nRunning validation regressions...")
all_results = []
for idx in index_cols:
    for outcome in PLACES_OUTCOMES:
        all_results.append(run_models(df, outcome, idx, state_col, ruca_col))

results = pd.concat(all_results, ignore_index=True)
results = results.sort_values(["outcome", "index", "model", "ruca_level"], na_position="last")

print("\n" + "="*90)
print("TOPLINE RESULTS (first 120 rows)")
print("="*90)
print(results.head(120).to_string(index=False))

# ============================================================
# 8) SAVE OUTPUTS
# ============================================================

if SAVE_OUTPUTS:
    out_indices = OUT_DIR / "tract_indices_merged.csv"
    out_results = OUT_DIR / "places_validation_results.csv"

    keep_cols = ["GEOID", state_col] + ([ruca_col] if ruca_col else [])
    keep_cols += [ndvi_col, evi_col, tcc_col] + NLCD_SELECTED + index_cols + PLACES_OUTCOMES
    keep_cols = [c for c in keep_cols if c in df.columns]

    df[keep_cols].to_csv(out_indices, index=False)
    results.to_csv(out_results, index=False)

    print("\nSaved outputs:")
    print(f"  - {out_indices}")
    print(f"  - {out_results}")

print("\nDone.")



Loading indicators...
  NDVI: tract_ndvi_summer_2021_2025_summary.csv  col=mean_ndvi  rows=85,187
  EVI : tract_evi_summer_2021_2025_summary.csv   col=mean_evi   rows=85,187
  TCC : tract_tcc_nlcd_2021_2023_summary.csv   col=tcc_mean_2021_2023   rows=85,187
Biophysical merged rows: 85,187

Loading NLCD + PLACES...
Using state col: StateAbbr
No RUCA detected in PLACES (pooled models only).

NLCD columns selected:
  - pct_deciduous_forest
  - pct_evergreen_forest
  - pct_mixed_forest
  - pct_shrub_scrub
  - pct_grassland_herbaceous
  - pct_emergent_herbaceous_wetlands
  - pct_woody_wetlands
  - pct_pasture_hay
  - pct_developed_open_space

Full merged rows: 85,187
Rows with PLACES present (OBESITY_CrudePrev): 77,939

Scaled diagnostics (first 12 scaled cols)
- mean_ndvi_01: n=83,509  min=0.0000  p1=0.0000  p50=0.6124  p99=1.0000  max=1.0000
- mean_evi_01: n=83,509  min=0.0000  p1=0.0000  p50=0.5277  p99=1.0000  max=1.0000
- tcc_mean_2021_2023_01: n=83,509  min=0.0000  p1=0.0000  p50=0.2

In [ ]:
#!/usr/bin/env python3
# ------------------------------------------------------------
# purpose: Merge 2020 RUCA tract codes (USDA ERS) into your saved
#          tract_indices_merged.csv, then rerun validation regressions.
# ------------------------------------------------------------

import warnings
from pathlib import Path
from typing import Optional, List

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

warnings.filterwarnings("ignore")

# ============================================================
# CONFIG
# ============================================================

ROOT = Path("/mnt/sda-21.8/bdevoe/greenspace")

INDICES_FILE = ROOT / "processed" / "index_validation" / "tract_indices_merged.csv"
OUT_RESULTS  = ROOT / "processed" / "index_validation" / "places_validation_results_with_ruca.csv"
OUT_MERGED   = ROOT / "processed" / "index_validation" / "tract_indices_merged_with_ruca.csv"

RUCA_LOCAL   = ROOT / "resources" / "RUCA-codes-2020-tract.csv"

PLACES_OUTCOMES = [
    "OBESITY_CrudePrev",
    "LPA_CrudePrev",
    "MHLTH_CrudePrev",
    "PHLTH_CrudePrev",
]

STATE_COL = "StateAbbr"
RUCA_COL_OUT = "RUCA10"
MIN_STRAT_N = 300

# ============================================================
# HELPERS
# ============================================================

def standardize_geoid(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip()
    s = s.str.replace(r"\.0$", "", regex=True)
    s = s.str.replace(r"[^0-9]", "", regex=True)
    return s.str.zfill(11)

def find_first_existing_col(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    for c in candidates:
        if c in df.columns:
            return c
    return None

def read_csv_robust(path: Path) -> pd.DataFrame:
    for enc in ("utf-8", "utf-8-sig", "cp1252", "latin1"):
        try:
            return pd.read_csv(path, encoding=enc)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path, encoding="latin1", on_bad_lines="skip")

def run_models(df: pd.DataFrame, outcome: str, index_col: str, state_col: str, ruca_col: str) -> pd.DataFrame:
    rows = []

    d = df[[outcome, index_col, state_col, ruca_col]].copy()
    d = d.dropna(subset=[outcome, index_col, state_col, ruca_col])

    m1 = smf.ols(f"{outcome} ~ {index_col}", data=d).fit(cov_type="HC1")
    rows.append(("pooled_unadjusted", None, m1))

    m2 = smf.ols(f"{outcome} ~ {index_col} + C({state_col})", data=d).fit(cov_type="HC1")
    rows.append(("pooled_stateFE", None, m2))

    m3 = smf.ols(f"{outcome} ~ {index_col} + C({state_col}) + C({ruca_col})", data=d).fit(cov_type="HC1")
    rows.append(("pooled_stateFE_rucaFE", None, m3))

    for r in sorted(d[ruca_col].dropna().unique()):
        dr = d[d[ruca_col] == r].copy()
        if len(dr) < MIN_STRAT_N:
            continue
        ms = smf.ols(f"{outcome} ~ {index_col} + C({state_col})", data=dr).fit(cov_type="HC1")
        rows.append(("ruca_strat_stateFE", r, ms))

    out = []
    for model_name, ruca_level, m in rows:
        if index_col not in m.params.index:
            continue
        out.append({
            "index": index_col,
            "model": model_name,
            "ruca_level": ruca_level,
            "outcome": outcome,
            "estimate": float(m.params[index_col]),
            "std_error": float(m.bse[index_col]),
            "t": float(m.tvalues[index_col]),
            "p": float(m.pvalues[index_col]),
            "n": int(m.nobs),
            "r2": float(m.rsquared),
        })
    return pd.DataFrame(out)

# ============================================================
# 1) LOAD RUCA
# ============================================================

print(f"Loading RUCA: {RUCA_LOCAL}")
ruca = read_csv_robust(RUCA_LOCAL)

# Your RUCA file has TractFIPS20 and TractFIPS23; prefer 2020 tracts
geoid_col = find_first_existing_col(
    ruca,
    ["TractFIPS20", "TractFIPS23", "TractFIPS", "GEOID", "geoid", "FIPS", "fips"]
)
if geoid_col is None:
    raise ValueError(f"Could not find a GEOID/FIPS column in RUCA file. Columns: {list(ruca.columns)[:60]}")

primary_col = find_first_existing_col(
    ruca,
    ["PrimaryRUCA", "primaryruca", "Primary RUCA", "Primary_RUCA"]
)
if primary_col is None:
    raise ValueError(f"Could not find PrimaryRUCA in RUCA file. Columns: {list(ruca.columns)[:60]}")

ruca = ruca.rename(columns={geoid_col: "GEOID", primary_col: RUCA_COL_OUT}).copy()
ruca["GEOID"] = standardize_geoid(ruca["GEOID"])
ruca[RUCA_COL_OUT] = pd.to_numeric(ruca[RUCA_COL_OUT], errors="coerce")
ruca = ruca[ruca[RUCA_COL_OUT].between(1, 10)].copy()

print("\nRUCA loaded:")
print("  rows:", len(ruca))
print("  RUCA counts:")
print(ruca[RUCA_COL_OUT].value_counts().sort_index().to_string())

# ============================================================
# 2) LOAD INDICES + MERGE RUCA
# ============================================================

print(f"\nLoading indices: {INDICES_FILE}")
df = pd.read_csv(INDICES_FILE)
df["GEOID"] = standardize_geoid(df["GEOID"])

if STATE_COL not in df.columns:
    raise ValueError(f"Expected {STATE_COL} in indices file. Found: {df.columns.tolist()[:60]}")

df = df.merge(ruca[["GEOID", RUCA_COL_OUT]], on="GEOID", how="left")

print("\nMerged RUCA into indices:")
print("  total rows:", len(df))
print("  rows with RUCA:", df[RUCA_COL_OUT].notna().sum())

df.to_csv(OUT_MERGED, index=False)
print("Saved merged indices+RUCA:", OUT_MERGED)

# ============================================================
# 3) RUN REGRESSIONS WITH RUCA
# ============================================================

missing_outcomes = [o for o in PLACES_OUTCOMES if o not in df.columns]
if missing_outcomes:
    raise ValueError(f"Missing PLACES outcomes in merged df: {missing_outcomes}")

index_cols = [c for c in df.columns if c.startswith("green_")]
if not index_cols:
    raise ValueError("No index columns found (expected columns starting with 'green_').")

print("\nRunning validation regressions WITH RUCA...")
all_res = []
for idx in index_cols:
    for outcome in PLACES_OUTCOMES:
        all_res.append(run_models(df, outcome, idx, STATE_COL, RUCA_COL_OUT))

results = pd.concat(all_res, ignore_index=True)
results = results.sort_values(["outcome", "index", "model", "ruca_level"], na_position="last")

print("\n" + "="*90)
print("TOPLINE RESULTS (first 120 rows) — WITH RUCA")
print("="*90)
print(results.head(120).to_string(index=False))

results.to_csv(OUT_RESULTS, index=False)
print("\nSaved:", OUT_RESULTS)
print("\nDone.")
